In [121]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
import string
from nltk.corpus import wordnet
tw_token = TweetTokenizer(strip_handles=False)

In [122]:
project = pd.read_csv('0000000000002747_training_twitter_x_y_train.csv')
test = pd.read_csv('0000000000002747_test_twitter_x_test.csv')
set(project['airline'])

{'American', 'Delta', 'Southwest', 'US Airways', 'United', 'Virgin America'}

In [123]:
from nltk.corpus import stopwords

In [124]:
stop = stopwords.words('english')

In [125]:
punctuations = list(string.punctuation)
stop = stop + punctuations
l = ['@SouthwestAir', '@united', '@USAirways', '@JetBlue', '@AmericanAir', '@VirginAmerica']
stop = stop + l
stop.remove("isn't")

In [126]:
#stop

In [127]:
x_train = project['text']
x_test = test['text']
y_train = project['airline_sentiment']

In [128]:
x_train = list(x_train)
x_test = list(x_test)
y_train = list(y_train)


In [129]:
documents = []
for i in range(len(x_train)):
    documents.append((x_train[i], y_train[i]))


In [130]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [131]:
#documents

In [132]:
def get_simple_pos(tag) :
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else :
        return wordnet.NOUN

In [133]:
from nltk import pos_tag
pos = pos_tag("Hello")
pos[0][1]

'NNP'

In [134]:
def clean_tweet(tweet) :
    output_words = []
    words = tw_token.tokenize(tweet)
    #words = word_tokenize(tweet)
    for w in words:
        if w.lower() not in stop:
            if w.startswith('#') :
                w = w[1:]
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
                
    return output_words

In [135]:
Documents = [(clean_tweet(document), category) for document, category in documents]

In [136]:
dist = nltk.FreqDist(all_words)

NameError: name 'all_words' is not defined

In [ ]:
vocab_with_frequency = dist.most_common(3000)
vocab = [i[0] for i in vocab_with_frequency]

In [ ]:
def get_features(document):
    words = word_tokenize(document)
    features = {}
    for w in vocab:
        features[w] = (w in words)
    return features

In [ ]:
training_data = [(get_features(d), t) for d, t in documents]

In [ ]:
#clf = nltk.NaiveBayesClassifier.train(training_data)

In [ ]:
#nltk.classify.accuracy(clf, training_data)

In [ ]:
#nltk.classify.accuracy(sklearn_svc, training_data)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

In [ ]:
#sklearn_rfc = SklearnClassifier(RandomForestClassifier())

In [ ]:
#sklearn_rfc.train(training_data)

In [ ]:
#nltk.classify.accuracy(sklearn_rfc, training_data)

In [137]:
from sklearn.feature_extraction.text import CountVectorizer

In [138]:
texts = [" ".join(document) for document, category in Documents]
categories = [category for document, category in Documents]


In [139]:
#texts

In [159]:
count_vec = CountVectorizer(max_features=2000, strip_accents = 'ascii', binary = 'boolean',  tokenizer = tw_token.tokenize)
#count_vec = CountVectorizer( max_features=2627, max_df = 0.6)

In [160]:
x_train_transformed = count_vec.fit_transform(texts, categories)

In [161]:
x_test_transformed = count_vec.transform(x_test)

In [162]:
type(texts)

list

In [163]:
type(x_test_transformed)

scipy.sparse.csr.csr_matrix

In [164]:
type(x_train_transformed)

scipy.sparse.csr.csr_matrix

In [165]:
from sklearn.ensemble import RandomForestClassifier

In [166]:
rfc = RandomForestClassifier()

In [167]:
rfc.fit(x_train_transformed, categories)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [168]:
y_pred = rfc.predict(x_test_transformed)

In [169]:
y_pred

array(['negative', 'neutral', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype='<U8')

In [170]:
np.savetxt('answer.csv', y_pred ,fmt = '%s', delimiter = ',')

In [171]:
from sklearn.naive_bayes import MultinomialNB

In [172]:
mnb = MultinomialNB(alpha = 0.186 )

In [173]:
mnb.fit(x_train_transformed, categories)

MultinomialNB(alpha=0.186, class_prior=None, fit_prior=True)

In [174]:
y_pred1 = mnb.predict(x_test_transformed)

In [175]:
np.savetxt('answer1.csv', y_pred1 ,fmt = '%s', delimiter = ',')

In [59]:
from sklearn.ensemble import GradientBoostingClassifier

In [60]:
gbc = GradientBoostingClassifier()

In [824]:
gbc.fit(x_train_transformed, categories)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [825]:
y_pred2 = gbc.predict(x_test_transformed)

In [826]:
np.savetxt('answer2.csv', y_pred2 ,fmt = '%s', delimiter = ',')

In [827]:
from sklearn.svm import SVC

In [828]:
svc = SVC()

In [829]:
svc.fit(x_train_transformed, categories)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [830]:
y_pred3 = svc.predict(x_test_transformed)

In [832]:
np.savetxt('answer3.csv', y_pred3 ,fmt = '%s', delimiter = ',')

In [833]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [870]:
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1,2), max_df=0.8, min_df=0.1)

In [871]:
x_train_tfidf = tfidf.fit_transform(texts, categories)

In [872]:
x_test_tfidf = tfidf.transform(x_test)

In [873]:
mnb.fit(x_train_tfidf, categories)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [874]:
y_pred4 = mnb.predict(x_test_tfidf)

In [875]:
np.savetxt('answer.csv', y_pred4 ,fmt = '%s', delimiter = ',')